In [0]:
from pyspark.sql import functions as F

In [0]:
# 1. Process sales_customers

df_customers = spark.table("samples.bakehouse.sales_customers")

df_customers_staging = (
    df_customers
    .dropDuplicates(["customerID"])
    .withColumn("phone_number", F.regexp_replace("phone_number", "[^0-9]", ""))
    .withColumn("email_address", F.lower(F.col("email_address")))
    .withColumn("load_date", F.current_timestamp())
)
df_customers_staging.write.format("delta").mode("overwrite").save("/mnt/staging/sales_customers")

In [0]:
# 2. Process sales_franchises
df_franchises = spark.table("samples.bakehouse.sales_franchises")

df_franchises_staging = df_franchises.withColumn("load_date", F.current_timestamp())

df_franchises_staging.write.format("delta").mode("overwrite").save("/mnt/staging/sales_franchises")

In [0]:
# 3. Process sales_suppliers
df_suppliers = spark.table("samples.bakehouse.sales_suppliers")

df_suppliers_staging = (
    df_suppliers
    .withColumn("alergy", 
                F.when(
                    F.lower(F.col("ingredient")).rlike("pistachios|peanuts|poppy seeds"), "Y"
                ).otherwise("N")
               )
    .withColumn("load_date", F.current_timestamp())
)

df_suppliers_staging.write.format("delta").mode("overwrite").save("/mnt/staging/sales_suppliers")

In [0]:
# 4. Process sales_transactions
df_transactions = spark.table("samples.bakehouse.sales_transactions")

df_transactions_staging = df_transactions.withColumn("load_date", F.current_timestamp())

df_transactions_staging.write.format("delta").mode("overwrite").save("/mnt/staging/sales_transactions")

In [0]:
# 5. Process media_customer_reviews
df_reviews = spark.table("samples.bakehouse.media_customer_reviews")

df_reviews_staging = df_reviews.withColumn("load_date", F.current_timestamp())

df_reviews_staging.write.format("delta").mode("overwrite").save("/mnt/staging/media_customer_reviews")

In [0]:
# 6. Process media_gold_reviews_chunked
df_gold_reviews = spark.table("samples.bakehouse.media_gold_reviews_chunked")

df_gold_reviews_staging = df_gold_reviews.withColumn("load_date", F.current_timestamp())

df_gold_reviews_staging.write.format("delta").mode("overwrite").save("/mnt/staging/media_gold_reviews_chunked")